In [414]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
c = np.loadtxt('MacroSeries.txt', delimiter=",",usecols=0)
k = np.loadtxt('MacroSeries.txt', delimiter=",",usecols=1)
w = np.loadtxt('MacroSeries.txt', delimiter=",",usecols=2)
r = np.loadtxt('MacroSeries.txt', delimiter=",",usecols=3)
data=np.loadtxt('MacroSeries.txt', delimiter=",")
df= DataFrame(data, columns=['Consumption','Capital','Wage','Interest Rate'])
df.describe()

,Consumption,Capital,Wage,Interest Rate
count,1.000000e+02,1.000000e+02,1.000000e+02,100.000000
mean,1.052085e+07,7.472545e+06,1.044521e+07,1.016947
std,2.387348e+06,1.686713e+06,2.370184e+06,0.101594
min,7.198780e+06,5.123678e+06,7.147026e+06,0.762142
25%,8.899324e+06,6.334027e+06,8.835344e+06,0.947913
50%,1.002475e+07,7.135045e+06,9.952684e+06,1.021391
75%,1.169416e+07,8.299804e+06,1.161009e+07,1.077150
max,1.856368e+07,1.321256e+07,1.843022e+07,1.333941


In [406]:
##1(a)

#AR(1) Normal Distribution
def ar1_norm1_pdf(k, w, alpha, rho, sigma):
    z=-alpha*np.log(k)+np.log(w)-np.log(1-alpha)     #Generating z_t sequence with (k,w)
    zt1=np.append(z[0],z[0:(len(z)-1)])                     #Generating z_t-1 sequence
    mu_z=rho*zt1+(1-rho)*z[0]                       #Generating mu_z sequence
    pdf_vals    = ((1/(sigma * np.sqrt(2 * np.pi)) *np.exp( - (z - mu_z)**2 / (2 * sigma**2))))
    return pdf_vals

#Log Likelihood Function of ç Distribution
def log_lik_ar1norm1(k, w, alpha, rho, sigma):
    pdf_vals = ar1_norm1_pdf(k, w, alpha, rho, sigma)
    ln_pdf_vals = np.log(pdf_vals)
    log_lik_val = ln_pdf_vals.sum()
    return log_lik_val

#Criterion Function of AR(1) Normal Distribution
def critar11(params, *args):
    alpha, rho, sigma = params
    k,w = args
    log_lik_val = log_lik_ar1norm1(k, w, alpha, rho, sigma)
    neg_log_lik_val = -log_lik_val
    return neg_log_lik_val

#MLE Estimates and Inverse Hessian Matrix
import scipy.optimize as opt
alpha1_init = 0.01 # alpha in [0,1] 
rho1_init = 0.3  # rho in [-1,1]
sig1_init = 0.1   # sigma > 0
mle1_args = (k,w) # Using (k,w)
params1_init = np.array([alpha1_init, rho1_init, sig1_init])
results1 = opt.minimize(critar11, params1_init, args =(mle1_args),method='L-BFGS-B', bounds=((1e-10, None),(1e-10, None),(1e-10, None)))
alpha1_MLE, rho1_MLE, sigma1_MLE = results1.x
mu1_MLE=-alpha1_MLE*np.log(k[0])+np.log(w[0])-np.log(1-alpha1_MLE)
print('alpha_MLE=', alpha1_MLE, ' rho_MLE=', rho1_MLE,' mu_MLE=',mu1_MLE,' sigma_MLE=',sigma1_MLE)
results1
vcv_mle11 = results1.hess_inv
vcv_mle12 = results1.hess_inv.matmat(np.eye(3)) #Hessian Matrix
success = results1.success
print('Success = ', success)
print('VCV result = ',vcv_mle11) 
print(vcv_mle12) 

alpha_MLE= 0.43111312429  rho_MLE= 0.739335275103  mu_MLE= 9.88519752067  sigma_MLE= 0.092029906636
Success =  True
VCV result =  <3x3 LbfgsInvHessProduct with dtype=float64>
[[  2.66459841e+00  -2.83001397e+00  -4.65954487e-02]
 [ -2.83001397e+00   3.01494125e+00   4.96640296e-02]
 [ -4.65954487e-02   4.96640296e-02   8.65491128e-04]]


In [411]:
##1(b)

#AR(1) Normal Distribution
def ar1_norm2_pdf(k, r, alpha, rho, sigma):              
    z=(1-alpha)*np.log(k)-np.log(alpha)+np.log(r)     #Generating z_t sequence with (k,r)
    zt1=np.append(z[0],z[0:(len(z)-1)])                       #Generating z_t-1 sequence
    mu_z=rho*zt1+(1-rho)*z[0]                         #Generating mu_z sequence
    pdf_vals    = ((1/(sigma * np.sqrt(2 * np.pi)) *np.exp( - (z - mu_z)**2 / (2 * sigma**2))))
    return pdf_vals

#Log Likelihood Function of ç Distribution
def log_lik_ar1norm2(k, r, alpha, rho, sigma):
    pdf_vals = ar1_norm2_pdf(k, r, alpha, rho, sigma)
    ln_pdf_vals = np.log(pdf_vals)
    log_lik_val = ln_pdf_vals.sum()
    return log_lik_val

#Criterion Function of AR(1) Normal Distribution
def critar12(params, *args):
    alpha, rho, sigma = params
    k,r = args
    log_lik_val = log_lik_ar1norm2(k, r, alpha, rho, sigma)
    neg_log_lik_val = -log_lik_val
    return neg_log_lik_val

#MLE Estimates and Inverse Hessian Matrix
import scipy.optimize as opt
alpha2_init = 0.5  
rho2_init = 0.8     
sig2_init = 0.05    
mle2_args = (k,r)   # Using k and r 
params2_init = np.array([alpha2_init, rho2_init, sig2_init])
results2 = opt.minimize(critar12, params2_init, args =(mle2_args),method='L-BFGS-B', bounds=((1e-10, None),(1e-10, None),(1e-10, None)))
alpha2_MLE, rho2_MLE, sigma2_MLE = results2.x
mu2_MLE=(1-alpha2_MLE)*np.log(k[0])+np.log(r[0])-np.log(alpha2_MLE)

print('alpha_MLE=', alpha2_MLE, ' rho_MLE=', rho2_MLE,' mu_MLE=',mu2_MLE,' sigma_MLE=',sigma2_MLE)
results2
vcv_mle21 = results2.hess_inv
vcv_mle22 = results2.hess_inv.matmat(np.eye(3)) #Hessian Matrix
success = results2.success
print('Success = ', success)
print('VCV result = ',vcv_mle21) 
print(vcv_mle22) 

alpha_MLE= 0.431108047377  rho_MLE= 0.739339440909  mu_MLE= 9.83982770471  sigma_MLE= 0.092029743968
Success =  True
VCV result =  <3x3 LbfgsInvHessProduct with dtype=float64>
[[ 0.89664047 -0.92143587 -0.0451172 ]
 [-0.92143587  0.95537103  0.04686481]
 [-0.0451172   0.04686481  0.00238014]]


In [413]:
##1(c)
#Using equation (4), we get 'z_t = -ln(alpha) + (1-alpha)*ln(k_t)' when r_t = 1.
from scipy.integrate import quad
def ar1_norm3_pdf(r, alpha, rho, mu, sigma):
    z=np.log(r)+(1-alpha)*np.log(75000000)-np.log(alpha)  #Generating z_t sequence with (k=75000000,r)
    zt_1=10                                             #z_(t-1)=10
    mu_z=rho*zt_1+(1-rho)*mu                            #Generating mu_z sequence
    pdf_vals    = ((1/(sigma * np.sqrt(2 * np.pi)) *np.exp( - (z - mu_z)**2 / (2 * sigma**2))))
    return pdf_vals
def integrand1(x):
    return ar1_norm3_pdf(r=x, alpha=alpha1_MLE, rho=rho1_MLE, mu=mu1_MLE, sigma=sigma1_MLE)
prob1 = quad(integrand1,0, 1)                           #Intergrating r from 0 to 1, given k=25000 and z_(t-1)=10
print('The probability that r>1 is', 1-prob1[0])

The probability that r>1 is 0.6935497705374554
